In [ ]:
import multiprocessing
import tqdm

import numpy as np
import scipy.stats as st
import scipy
import numba
import math

#import biocircuits

# Plotting modules
import bokeh.io
import bokeh.plotting
from bokeh.models import LinearColorMapper, ColorBar
#from bokeh.io import export_svgs

bokeh.io.output_notebook()

# Line profiler (can install with conda install line_profiler)
# %load_ext line_profiler

# Analysis of molecular classifiers under shared resources

Molecular sequestration can realize a single node that process the sum weighted inputs and feed it into a smooth ReLU activation function. Sigma factors are a great candidates to implement the biomolecular neural networks. However, multiple sigma factors will compete for the core DNApol. Therefore, how will this interaction affect the deicision boundaries?

To answer this question, we will develop a mathematical model that incorporate the competition for resoures, and quantify the effect of compeition. Moreover, to identify simple design rules for molecular classifiers.

Next, we will model multi layer neural networks and characterize the input-output behavior of molecular classsifiers.

Our hypothesis is that compeition for resouces can add a bias. This bias could depend as a fucntion of the concentration of the competitive species in a non-linear way. We will identify under what condition, we can still predict the linear classifiers and compose multi-layer neural networks.


## Chemical Reactions

\begin{align}
\emptyset \xrightarrow[]{\alpha_1} S_1 && S_1  \xrightarrow{\delta} \emptyset \\
\emptyset \xrightarrow[]{\beta_1} A_1 && A_1  \xrightarrow{\delta} \emptyset \\
S_1 + A_1 \xrightarrow{\gamma_1} \emptyset &&  S_1 + C \xrightarrow{\gamma_2} C_1
\end{align}

## Mathematical Modeling

\begin{align}
  \dot s_1 &= \alpha_1 - \delta s_1 - \gamma_1 s_1a_1 - \gamma_2 s_1c\\
  \dot a_1 &= \beta_1 - \delta a_1 - \gamma_1 s_1a_1 \\
  \dot c_1 &= \gamma_2 s_1c - \delta c_1
\end{align}
with mass conservation $c + c_1 = c^{tot}$.
## Steady state analysis

\begin{align}
  \bar a_1 &= \frac{\alpha_1 - \delta \bar s_1 - \gamma_2 \bar s_1 \bar c}{\gamma_1 \bar  s_1} = \frac{\beta_1}{\gamma_1 \bar s_1 + \delta} \\
  \bar a_2 &= \frac{\alpha_2 - \delta \bar s_2 - \gamma_2 \bar s_2 \bar c}{\gamma_1 \bar  s_2} = \frac{\beta_2}{\gamma_1 \bar s_2 + \delta} \\
  \bar c_1 &= \frac{\gamma_2}{\delta} \bar s_1 \bar c \\
  \bar c_2 &= \frac{\gamma_2}{\delta} \bar s_2 \bar c \\
\end{align}

Then, we can use the mass conservation of $c$ and find that
\begin{align}
  \bar c = \frac{1}{1 + \frac{\gamma_2}{\delta} s_1 + \frac{\gamma_2}{\delta} s_2}c^{tot}
\end{align}

Then, we can find that
\begin{align}
  \bar c_1 &= \frac{s_1}{\frac{\delta}{\gamma_2} +  s_1 + s_2}c^{tot} \\
  \bar c_2 &= \frac{s_2}{\frac{\delta}{\gamma_2} +  s_1 + s_2}c^{tot} \\
\end{align}

In [ ]:
def fun_SS_only(a, b, g1, g2, d, ct):
    A = - a/d + b/d  + d/g1
    B = -a/g1

    coef = np.array([1, A, B])
    rp = np.roots(coef)
    return rp[(rp.imag == 0) & (rp>=0)].real[:]

In [ ]:
a_v = np.linspace(0,1,51)
b = 0.4
g1 = [10,100,1000]
g2 = 1.0
d = 1
ct = 1/5

s_v = np.empty_like(a_v)

palette = bokeh.palettes.grey(5)[::-1]

# Set up plots
fig_size = [150, 115] # for paper size (144, 115) # for larger size (144,144)
#fig_size = [360, 275] # to visualize
x_range = (0, 1)
y_range = (0, 1)
plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range,y_range=y_range
                          ))
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"




for j, g1i in enumerate(g1):
    s_v = np.empty_like(a_v)
    for i, a in enumerate(a_v):
        s_v[i] = fun_SS_only(a, b, g1i, g2, d, ct)

    plots[0].line(a_v, s_v, line_width=2, color=palette[j+1])


plots[0].output_backend = "svg"
# bokeh.io.export_svgs(p, filename="test.svg")

bokeh.io.show(bokeh.layouts.row(plots))

<ipython-input-3-8b4c12dcd371>:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  s_v[i] = fun_SS_only(a, b, g1i, g2, d, ct)


In [ ]:
a_v = np.linspace(0,2,51)
b = [0.2, 0.4, 0.6]
g1 = 100
g2 = 1.0
d = 1
ct = 1/5

s_v = np.empty_like(a_v)

palette = bokeh.palettes.grey(5)[::-1]

# Set up plots
fig_size = [150, 115] # for paper size (144, 115) # for larger size (144,144)
#fig_size = [360, 275] # to visualize
x_range = (-1, 1)
y_range = (0, 1)
plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"




for j, bi in enumerate(b):
    s_v = np.empty_like(a_v)
    for i, a in enumerate(a_v):
        s_v[i] = fun_SS_only(a, bi, g1, g2, d, ct)

    plots[0].line(a_v-bi, s_v, line_width=2, color=palette[j+1])


plots[0].output_backend = "svg"
# bokeh.io.export_svgs(p, filename="test.svg")

bokeh.io.show(bokeh.layouts.row(plots))

<ipython-input-4-e12b06020987>:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  s_v[i] = fun_SS_only(a, bi, g1, g2, d, ct)


In [ ]:
def fun_SS_s1(a, b, g1, g2, d, ct):
    A = -a/d + b/d + ct + d/g1 + d/g2
    B = -a/g1 - a/g2 + b/g2 + d*ct/g1 + d**2/g1/g2
    C = -d*a/g1/g2

    coef = np.array([1, A, B, C])
    rp = np.roots(coef)
    return rp[(rp.imag == 0) & (rp>=0)].real[:]

In [ ]:
a_v = np.linspace(0,1,51)
b = 0.4
g1 = [10,100,1000]
g2 = 10
d = 1
ct = 1/5

s_v = np.empty_like(a_v)

palette = bokeh.palettes.Oranges5[::-1]

# Set up plots
fig_size = [150, 115] # for paper size (144, 115) # for larger size (144,144)
#fig_size = [360, 275] # to visualize
x_range = (0, 1)
y_range = (0, 1)
plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"




for j, g1i in enumerate(g1):
    s_v = np.empty_like(a_v)
    for i, a in enumerate(a_v):
        s_v[i] = fun_SS_s1(a, b, g1i, g2, d, ct)

    plots[0].line(a_v, s_v/(s_v + d/g2), line_width=2, color=palette[j+1])


plots[0].output_backend = "svg"
# bokeh.io.export_svgs(p, filename="test.svg")

bokeh.io.show(bokeh.layouts.row(plots))

<ipython-input-6-23953da7eae7>:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  s_v[i] = fun_SS_s1(a, b, g1i, g2, d, ct)


In [ ]:
a_v = np.linspace(0,5,51)
b = [0.2, 0.4, 0.6]
g1 = 100
g2 = 10
d = 1
ct = 1/5

s_v = np.empty_like(a_v)

palette = bokeh.palettes.Oranges5[::-1]

# Set up plots
fig_size = [150, 115] # for paper size (144, 115) # for larger size (144,144)
#fig_size = [360, 275] # to visualize
x_range = (-1, 1)
y_range = (0, 1)
plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"




for j, bi in enumerate(b):
    s_v = np.empty_like(a_v)
    for i, a in enumerate(a_v):
        s_v[i] = fun_SS_s1(a, bi, g1, g2, d, ct)

    plots[0].line(a_v-bi, s_v/(s_v + d/g2), line_width=2, color=palette[j+1])


plots[0].output_backend = "svg"
# bokeh.io.export_svgs(p, filename="test.svg")

bokeh.io.show(bokeh.layouts.row(plots))

<ipython-input-11-fe7e88e09638>:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  s_v[i] = fun_SS_s1(a, bi, g1, g2, d, ct)


In [ ]:
def Sequestration_rhs(x,t,a1,b1,a2,b2,g1,g2,d,ct):
    S1, A1, S2, A2, C1, C2 = x
    C = ct - C1 - C2
    return np.array(
        [
            a1 - d*S1 - g1*A1*S1 - g2*S1*C,
            b1 - d*A1 - g1*A1*S1,
            a2 - d*S2 - g1*A2*S2 - g2*S2*C,
            b2 - d*A2 - g1*A2*S2,
            g2*S1*C - d*C1,
            g2*S2*C - d*C2,
        ]
    )

x0 = np.array([0., 0., 0., 0., 0., 0.])
a1 = 1
b1 = 0.5
a2 = 0
b2 = 0

g1 = 100
g2 = 10

d = 1
ct = 1/5

t = np.linspace(0,15,51)
x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(a1, b1, a2, b2,  g1, g2, d, ct))
x = x.transpose()

# Set up plots
fig_size = [144, 144] # for paper size (144, 115) # for larger size (144,144)
x_range = (0, 6)
y_range = (0, 0.61)

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1]
                                       #,x_range=x_range,y_range=y_range
                                       ))
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"


plots[0].line(t,x[0,:], line_width=2, color="black")
plots[0].line(t,x[1,:], line_width=2, color="orange")

plots[0].circle(t[10:41:10],x[0,10:41:10], line_width=2, color="black")


bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
N = 51
tN = 51
x1 = np.linspace(0,2,N)
x2 = 0.4

g1 = 100
g2 = 10
d = 1
ct = 1/5

a2v = [0, 0.5, 1.]
x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)



output1 = np.zeros((N,3,tN))
output2 = np.zeros((N,3,tN))
for j, a2j in enumerate(a2v):
    for i, x1i in enumerate(x1):
        x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(x1i, x2, a2j, b2,  g1, g2, d, ct))
        output1[i,j,:] = x.transpose()[0] # 0 (s1) 4 (c1)
        output2[i,j,:] = x.transpose()[4] # 0 (s1) 4 (c1)

# Set up plots
#fig_size = (150,115)
fig_size = (130,115)
x_range = (-1,1)
y_range = (0,1)
palette = bokeh.palettes.Blues5[::-1]

plots = []
for i in range(4):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate((10,20,30,50)):
    for z, itr2 in enumerate(a2v):
        plots[k].line(x1-x2, output2[:,z,itr]/ct, line_width=2, color=palette[z+1])


plots[3].output_backend = "svg"
#bokeh.io.show(bokeh.layouts.row(plots))
bokeh.io.show(bokeh.layouts.gridplot(plots,ncols=4)) # Top S1, and botton C1

#Plotting for papaer
fig_size = (150,115)
plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"



for z, itr2 in enumerate(a2v):
        plots[0].line(x1-x2, output2[:,z,50]/ct, line_width=2, color=palette[z+1])


plots[0].output_backend = "svg"
# bokeh.io.export_svgs(p, filename="test.svg")

bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
N = 51
tN = 51
x1 = np.linspace(0,2,N)
x2 = 0.4

g1 = 100
g2 = 10
d = 1
ct = 1/5

b2v = [0, 0.5, 1.]
a2 = 0.5
x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)



output1 = np.zeros((N,3,tN))
output2 = np.zeros((N,3,tN))
for j, b2j in enumerate(a2v):
    for i, x1i in enumerate(x1):
        x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(x1i, x2, a2, b2j,  g1, g2, d, ct))
        output1[i,j,:] = x.transpose()[0] # 0 (s1) 4 (c1)
        output2[i,j,:] = x.transpose()[4] # 0 (s1) 4 (c1)

# Set up plots
#fig_size = (150,115)
fig_size = (130,115)
x_range = (-1,1)
y_range = (0,1)
palette = bokeh.palettes.Blues5[::-1]

plots = []
for i in range(4):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate((10,20,30,50)):
    for z, itr2 in enumerate(a2v):
        plots[k].line(x1-x2, output2[:,z,itr]/ct, line_width=2, color=palette[z+1])


plots[3].output_backend = "svg"
#bokeh.io.show(bokeh.layouts.row(plots))
bokeh.io.show(bokeh.layouts.gridplot(plots,ncols=4)) # Top S1, and botton C1

#Plotting for papaer
fig_size = (150,115)
plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"



for z, itr2 in enumerate(a2v):
        plots[0].line(x1-x2, output2[:,z,50]/ct, line_width=2, color=palette[z+1])


plots[0].output_backend = "svg"
# bokeh.io.export_svgs(p, filename="test.svg")

bokeh.io.show(bokeh.layouts.row(plots))

## Root analysis for finding steady-state solution of c̄

---



In [ ]:
a = 0.2
b = 0.4
d = 0.1
r = 1.
rv = [0.01, 0.1, 1.]

cv = np.linspace(-0.1,1.1,151)
F1 = cv*(cv - (a-b+d))*(cv-1)
F2 = d*cv + (cv - 1)*((cv - a)*(cv - 1) + d*cv)*r

fig_size = (130,115)
fig_size = (400,250)
x_range = (-0.1,1.1)
y_range = (-0.12,0.12)
palette = bokeh.palettes.Blues5[::-1]

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range,
                         y_range=y_range),
                        )


plots[0].line(cv, F1, line_width=2, color="black")
#plots[0].line(cv, d*cv, line_width=2, color="grey")

for z, rk in enumerate(rv):
    F2 = d*cv + (cv - 1)*((cv - a)*(cv - 1) + d*cv)*rk
    plots[0].line(cv, F2, line_width=2, color=palette[z+1])

plots[0].output_backend = "svg"
# bokeh.io.export_svgs(p, filename="test.svg")

bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
a = 0.4
b = 0.2
d = 0.1
r = 1.
rv = [0.01, 0.1, 1.]

cv = np.linspace(-0.1,1.1,151)
F1 = cv*(cv - (a-b+d))*(cv-1)
F2 = d*cv + (cv - 1)*((cv - a)*(cv - 1) + d*cv)*r

fig_size = (130,115)
fig_size = (400,250)
x_range = (-0.1,1.1)
y_range = (-0.12,0.12)
palette = bokeh.palettes.Blues5[::-1]

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range,
                         y_range=y_range),
                        )


plots[0].line(cv, F1, line_width=2, color="black")
#plots[0].line(cv, d*cv, line_width=2, color="grey")

for z, rk in enumerate(rv):
    F2 = d*cv + (cv - 1)*((cv - a)*(cv - 1) + d*cv)*rk
    plots[0].line(cv, F2, line_width=2, color=palette[z+1])

plots[0].output_backend = "svg"
# bokeh.io.export_svgs(p, filename="test.svg")

bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
a = 1.3
b = 0.2
d = 0.1
r = .1
rv = [0.01, 0.1, 1.]

cv = np.linspace(-0.1,1.1,151)
F1 = cv*(cv - (a-b+d))*(cv-1)
F2 = d*cv + (cv - 1)*((cv - a)*(cv - 1) + d*cv)*r

fig_size = (130,115)
fig_size = (400,250)
x_range = (-0.1,1.1)
y_range = (-0.12,0.12)
palette = bokeh.palettes.Blues5[::-1]

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range,
                         y_range=y_range),
                        )


plots[0].line(cv, F1, line_width=2, color="black")
#plots[0].line(cv, d*cv, line_width=2, color="grey")

for z, rk in enumerate(rv):
    F2 = d*cv + (cv - 1)*((cv - a)*(cv - 1) + d*cv)*rk
    plots[0].line(cv, F2, line_width=2, color=palette[z+1])

plots[0].output_backend = "svg"
# bokeh.io.export_svgs(p, filename="test.svg")

bokeh.io.show(bokeh.layouts.row(plots))

## Singular perturbation of two variables

In [ ]:
a = 0.1
b = 0.2
xi = 0.01
r = .1
rv = [1, 0.1, .01]

cv = np.linspace(-0.1,1.1,151)

F0 = cv*(cv - a + b)*(cv-1)
F1 = cv**2*xi
F2 = (cv-1)**2*(cv-a)*r
F3 = (cv-1)*cv*xi*r
Q = F1 + F2 + F3

fig_size = (150*2,150)
#fig_size = (400,250)
x_range = (-0.1,1.1)
y_range = (-0.12,0.12)
palette = bokeh.palettes.Blues5[::-1]

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range,
                         y_range=y_range),
                        )


plots[0].line(cv, F0, line_width=2, color="black")
#plots[0].line(cv, d*cv, line_width=2, color="grey")

for z, rk in enumerate(rv):
    F1 = cv**2*xi
    F2 = (cv-1)**2*(cv-a)*rk
    F3 = (cv-1)*cv*xi*rk
    G = F1 + F2 + F3
    plots[0].line(cv, G, line_width=2, color=palette[z+1])

plots[0].output_backend = "svg"
# bokeh.io.export_svgs(p, filename="test.svg")

bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
a = 0.5
b = 0.2
xi = 0.01
r = .1
rv = [1, 0.1, .01]

cv = np.linspace(-0.1,1.1,151)

F0 = cv*(cv - a + b)*(cv-1)
F1 = cv**2*xi
F2 = (cv-1)**2*(cv-a)*r
F3 = (cv-1)*cv*xi*r
Q = F1 + F2 + F3

fig_size = (150*2,150)
x_range = (-0.1,1.1)
y_range = (-0.12,0.12)
palette = bokeh.palettes.Blues5[::-1]

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range,
                         y_range=y_range),
                        )


plots[0].line(cv, F0, line_width=2, color="black")
#plots[0].line(cv, d*cv, line_width=2, color="grey")

for z, rk in enumerate(rv):
    F1 = cv**2*xi
    F2 = (cv-1)**2*(cv-a)*rk
    F3 = (cv-1)*cv*xi*rk
    G = F1 + F2 + F3
    plots[0].line(cv, G, line_width=2, color=palette[z+1])

plots[0].output_backend = "svg"
# bokeh.io.export_svgs(p, filename="test.svg")

bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
a = 1.3
b = 0.2
xi = 0.01
r = .1
rv = [1, 0.1, .01]

cv = np.linspace(-0.1,1.1,151)

F0 = cv*(cv - a + b)*(cv-1)
F1 = cv**2*xi
F2 = (cv-1)**2*(cv-a)*r
F3 = (cv-1)*cv*xi*r
Q = F1 + F2 + F3

fig_size = (150*2,150)
x_range = (-0.1,1.1)
y_range = (-0.12,0.12)
palette = bokeh.palettes.Blues5[::-1]

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range,
                         y_range=y_range),
                        )


plots[0].line(cv, F0, line_width=2, color="black")
#plots[0].line(cv, d*cv, line_width=2, color="grey")

for z, rk in enumerate(rv):
    F1 = cv**2*xi
    F2 = (cv-1)**2*(cv-a)*rk
    F3 = (cv-1)*cv*xi*rk
    G = F1 + F2 + F3
    plots[0].line(cv, G, line_width=2, color=palette[z+1])

plots[0].output_backend = "svg"
# bokeh.io.export_svgs(p, filename="test.svg")

bokeh.io.show(bokeh.layouts.row(plots))